In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os

def studio_quality_enhancer(image_path):
    # 1. Load Image
    if not os.path.exists(image_path):
        print("Error: Image not found.")
        return None, None

    img = cv2.imread(image_path)
    if img is None:
        return None, None
        
    h, w = img.shape[:2]

    # 2. SUBJECT SEGMENTATION 
    # Using a fixed rectangle for subject focus
    mask = np.zeros(img.shape[:2], np.uint8)
    rect = (int(w*0.05), int(h*0.05), int(w*0.9), int(h*0.9))
    
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    # GrabCut isolates the human from background
    cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 3, cv2.GC_INIT_WITH_RECT)
    
    # Create mask: 1 for foreground, 0 for background
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    mask_3d = cv2.merge([mask2, mask2, mask2])

    # -- 3. BACKGROUND BLUR (PORTRAIT EFFECT) --
    blurred_bg = cv2.GaussianBlur(img, (99, 99), 0)
    # Combine sharp subject and blurred background
    bokeh_img = np.where(mask_3d == 1, img, blurred_bg)

    # -- 4. FACE CLARITY & SHARPNESS--
    sharp_img = cv2.detailEnhance(bokeh_img, sigma_s=10, sigma_r=0.15)

    # -- 5. STUDIO LIGHTING (CLAHE) --

    lab = cv2.cvtColor(sharp_img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8))
    l = clahe.apply(l)
    studio_light = cv2.cvtColor(cv2.merge((l, a, b)), cv2.COLOR_LAB2BGR)

    # -- 6. SKIN TEXTURE PRESERVATION (Bilateral Filter)--
    final_output = cv2.bilateralFilter(studio_light, 9, 75, 75)

    return img, final_output

# ------ RUNNING THE PIPELINE ------
# Here file is uploaded as 'myphoto.jpg'
target = '/content/myphoto.jpg'

if os.path.exists(target):
    original, result = studio_quality_enhancer(target)
    
    if result is not None:
        print("SUCCESS: Studio-quality portrait generated:)")
        
        # Display side-by-side
        h_disp, w_disp = 600, 450
        comparison = np.hstack((cv2.resize(original, (w_disp, h_disp)), 
                                cv2.resize(result, (w_disp, h_disp))))
        cv2_imshow(comparison)
else:
    print("Upload your image first!")